In [2]:
library(EnsDb.Hsapiens.v86)
library(BSgenome.Hsapiens.UCSC.hg38)
library(GenomicRanges)
library(data.table)

In [3]:
# Get genome length
genome_len <- 0
for (i in names(Hsapiens)[1:24]) {
    genome_len <- genome_len + length(Hsapiens[[i]])
}
genome_len

[1] 3088269832

### Below we create a gene annotation object from the Ensembl Homo Sapiens v86 gff3 annotation

In [26]:
#--- Use system (Unix) commands to extract exon locations from a hg38 .gff3 file
cat("\nFiltering hg38 gff3 file (Ensembl v86) for gene ranges... \n")
system(paste("grep ID=gene ../Data/humangenome/Homo_sapiens.GRCh38.86.gff3 > ./tmp")) # Write lines with gene in third column to a tmp file
# system(paste("grep biotype=protein_coding ./tmp > ./tmp2")) # If you want to filter for protein-coding genes only
# system(paste("awk '$3~/gene/{print}' ../Data/humangenome/Homo_sapiens.GRCh38.86.gtf > ./tmp")) # For gtf file instead of gff3
system(paste("cut -f1,4,5 ./tmp > ./tmp3")) # Extract columns 1, 4, 5 from tmp file, and write result to ../Data directory
cat("Reading gene location table...\n")
gann <- read.table("./tmp3") # Read exonic regions into table (chromName	start	end)
system(paste("rm ./tmp"))
# system(paste("rm ./tmp2"))
system(paste("rm ./tmp3"))
cat("Done")
names(gann) <- c('chrom','start','end')

tmp<-c()
gann <- GRanges(seqnames=gann$chrom,IRanges(gann$start,gann$end)) # Convert gann into GRanges object (bioconductor)
# gann <- reduce(gann) # Reduce to non-overlapping ranges
tmp$chrom <- as.vector(seqnames(gann)) # Next lines are for conversion back to data.frame
tmp$start <- start(gann)
tmp$end <- end(gann)
gann <- data.frame(tmp)
names(gann) <- c('chrom','start','end')
head(gann)
cat(paste0(toString(nrow(gann)),' genes'))
cat(paste0('\n\n',toString(sum(gann$end-gann$start)/genome_len),' of genome'))


Filtering hg38 gtf/gff3 file (Ensembl v86) for gene ranges... 
Reading gene location table...
Done

chrom,start,end
1,11869,14409
1,14404,29570
1,17369,17436
1,29554,31109
1,34554,36081
1,52473,53312


58051 genes

0.574797676876054 of genome

### Alternatively, we can create a gene annotation object from the EnsemblDb Homo Sapiens v86 Bioconductor database

In [28]:
gann <- genes(EnsDb.Hsapiens.v86)
gann <- data.table(chrom=as.vector(seqnames(gann)),start=start(gann),end=end(gann),geneSym=gann$gene_name)
head(gann)
cat(paste0(toString(nrow(gann)),' genes'))
cat(paste0('\n\n',toString(sum(gann$end-gann$start)/genome_len),' of genome'))

chrom,start,end,geneSym
1,11869,14409,DDX11L1
1,14404,29570,WASH7P
1,17369,17436,MIR6859-1
1,29554,31109,MIR1302-2
1,34554,36081,FAM138A
1,52473,53312,OR4G4P


63970 genes

0.614410695379937 of genome

### We can also filter the database for only genes with biotype of protein-coding

In [23]:
gann <- genes(EnsDb.Hsapiens.v86,filter=GeneBiotypeFilter('protein_coding'))
gann <- data.table(chrom=as.vector(seqnames(gann)),start=start(gann),end=end(gann),geneSym=gann$gene_name)
head(gann)
cat(paste0(toString(nrow(gann)),' genes'))
cat(paste0('\n\n',toString(sum(gann$end-gann$start)/genome_len),' of genome'))

chrom,start,end,geneSym
1,69091,70008,OR4F5
1,182393,184158,FO538757.2
1,184923,200322,FO538757.1
1,450740,451678,OR4F29
1,685716,686654,OR4F16
1,923928,944581,SAMD11


22285 genes

0.452765256297073 of genome

### After choosing the annotation, we count the L1 target sites in gene regions for each chromosome (as well as some other columns, see comments below)

In [ ]:
gene_counts <- 	array(0,dim=c(24,9)) # Allocate memory for a 2D array containing counts of gene insertion sites of each S-V type for each chromosome, and other information

#--- Loop through chromosome names
j<-1
for (i in names(Hsapiens)[1:24]){

	cat("\nProccessing ",i,"...")
	load(paste0("../Data/root_maps/",i,".rda")) # Load map file for current chromosome

	# Data objects containing indices of S-V sites for each category are labeled with the chromosome name.
	# Here we copy the data objects to a set of new names which can be used consistently in the following
    # code.
	map<-get(paste0(i,"Map"))	
        ict<-map$ict
        icl<-map$icl
        iot<-map$iot
        iol<-map$iol
        insites<-map$insites

	# Extract gene regions for the current chromosome
	chrno<-strsplit(i,"chr")[[1]][2] # Get the current chromosome number (or letter)
	gann_i <- gann[gann$chrom == chrno,] # Extract a subset of the 'gann' table for regions in the current chromosome

	# Fill in columns 1-4 of the row of the gene_counts array corresponding to current chromosome.
	# These columns contain the number of sites of each S-V category lying within genes 
	# Categories are in this order: closed-tight, closed-loose, open-tight, open-loose
	tmp     <-inrange(insites[ict[which(!is.na(ict[,1])),1],1],gann_i$start,gann_i$end) # Check if any Closed-Tight category sites are within the start-end range of gann_i
	tmp2    <-inrange(insites[ict[which(!is.na(ict[,2])),2],2],gann_i$start,gann_i$end) 
	gene_counts[j,1]<-length(which(tmp == TRUE)) + length(which(tmp2==TRUE)) 	  # Fill an element of the gene_counts table with the count
	tmp     <-inrange(insites[icl[which(!is.na(icl[,1])),1],1],gann_i$start,gann_i$end) # Check if any Closed-Tight category sites are within the start-end range of gann_i
    tmp2    <-inrange(insites[icl[which(!is.na(icl[,2])),2],2],gann_i$start,gann_i$end)
	gene_counts[j,2]<-length(which(tmp == TRUE)) + length(which(tmp2==TRUE))
	tmp     <-inrange(insites[iot[which(!is.na(iot[,1])),1],1],gann_i$start,gann_i$end) # Check if any Closed-Tight category sites are within the start-end range of gann_i
	tmp2    <-inrange(insites[iot[which(!is.na(iot[,2])),2],2],gann_i$start,gann_i$end)
	gene_counts[j,3]<-length(which(tmp == TRUE)) + length(which(tmp2==TRUE))
    tmp     <-inrange(insites[iol[which(!is.na(iol[,1])),1],1],gann_i$start,gann_i$end) # Check if any Closed-Tight category sites are within the start-end range of gann_i
	tmp2    <-inrange(insites[iol[which(!is.na(iol[,2])),2],2],gann_i$start,gann_i$end)
	gene_counts[j,4]<-length(which(tmp == TRUE)) + length(which(tmp2==TRUE))  


	# Fill in columns 5-8 of the row of the gene_counts array corresponding to current chromosome.
	# These columns contain the total number of sites of each category
	gene_counts[j,5]<-length(which(!is.na(ict)))
	gene_counts[j,6]<-length(which(!is.na(icl)))
	gene_counts[j,7]<-length(which(!is.na(iot)))
	gene_counts[j,8]<-length(which(!is.na(iol)))

	# The last column contains the gene fraction of the current chromosome
	tmp<-IRanges(gann_i$start,gann_i$end)
	gene_counts[j,9] <- sum(width(tmp))/length(Hsapiens[[i]])

	j<-j+1
}

In [ ]:
write.csv(gene_counts,file="~/jackgl/gene_counts.csv")